# Trabalho Final
Neste trabalho, vamos buscar o reconhecimento dos dígitos da Linguagem de Sinais. Para tal, vamos nos utilizar de Modelos conhecidos de Deep Learning e também nos aventurar na criação de próprios.

------

## Imports

In [1]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [2]:
import os
import numpy as np
from random import sample, seed
seed(42)
np.random.seed(42)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,15) # Make the figures a bit bigger

# Keras imports
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, Dense, Flatten
from keras.models import Model
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

import inf619utils

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Dataset
O dataset é composto por 10 classes (dígitos de 0 a 9) com aproximadamente 205 imagens por classe. 
O conjunto foi dividido em 60% para treinamento, 15% para validação e 20% para teste.
As imagens estão divididas em blocos por classe.

** IMPORTANTE NÃO ALTERAR O NOME/LOCAL DAS IMAGENS** 

In [3]:
datasetDir = "./Dataset"
nbClasses = 10
input_shape = (224,224,3)

train_files = {}
val_files = {}
test_files = {}

train_files, val_files, test_files = inf619utils.splitData(datasetDir, nbClasses)

In [4]:
#plot the images from imgList
def plotImagesFromBatch(imgList):
    for i in range(len(imgList)):
        plotImage(imgList[i])


# Se quiser visualizar algum bloco de imagens, descomentar as linhas abaixo
# inf619utils.plotImages(val_files)
# inf619utils.plotImages(train_files)
# inf619utils.plotImages(test_files)

In [5]:
trainSetSize = inf619utils.getDatasetSize(train_files)
valSetSize = inf619utils.getDatasetSize(val_files)
testSetSize = inf619utils.getDatasetSize(test_files)

print("# images in Train set: ", trainSetSize)
print("# images in Val set: ", valSetSize)
print("# images in Test set: ", testSetSize)

# images in Train set:  1242
# images in Val set:  309
# images in Test set:  511


In [6]:
for batch, labels in inf619utils.loadDatasetInBatches(train_files, batch_size=32, input_shape=input_shape, nbClasses=nbClasses, shouldAugmentData=True):
    print(batch.shape, labels.shape)
    #plotImagesFromBatch(batch)
    break

(32, 224, 224, 3) (32, 10)


## Definição do modelo 

In [7]:
model = ResNet50(input_shape=input_shape, classes=nbClasses, include_top=False)
model.summary()

D:\ProgramData\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation


# Modificação do modelo 

In [8]:
# Escolher a camada que será o ponto de partida 
x = model.output

#print([layer.name for layer in squeezeNetModel.layers])
#print("\n\nFreeze layers up until ", squeezeNetModel.layers[-20].name)

for layer in model.layers:
    layer.trainable = True#        layer.trainable = False

x = Flatten()(x)
x = Dense(10, activation='softmax')(x)


# Não se esqueça de definir o nome modelo, onde baseSqueezeNetModel 
# é o modelo base da Squeeze que vc definiu ali em cima
model = Model(model.inputs, x, name='resnet_new_adam')
#model2 = Model(squeezeNetModel.inputs, x, name='squeezenet_new_adadelta')
#model3 = Model(squeezeNetModel.inputs, x, name='squeezenet_new_sgd')

# Treinamento do Modelo

In [9]:
from keras.optimizers import SGD, Adam
#Compile o modelo
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])

import keras.callbacks as callbacks

tbCallBack = callbacks.TensorBoard(log_dir = "./logs_resnet")
tbEarly = callbacks.EarlyStopping(monitor='val_acc',min_delta=0,patience=10,verbose=0, mode='auto')
tbEarly100 = callbacks.EarlyStopping(monitor='acc', min_delta=0.0001, patience=3, mode='max')
tbModelChk = callbacks.ModelCheckpoint('.modelresnet_weights.hdf5', save_best_only=True, monitor='val_acc', mode='max')


In [10]:
#Definir tamanho do batch e número de épocas
batch_size = 4
epochs = 30

#Criação dos generators
trainGenerator_adam = inf619utils.loadDatasetInBatches(train_files, batch_size = batch_size, input_shape=input_shape, nbClasses=nbClasses, shouldAugmentData=True)
valGenerator_adam = inf619utils.loadDatasetInBatches(val_files, input_shape=input_shape, batch_size = batch_size, nbClasses=nbClasses)

#Fit nos dados
hist = model.fit_generator(trainGenerator_adam, 
                    steps_per_epoch= int(trainSetSize / batch_size), 
                    epochs = epochs,
                    validation_data = valGenerator_adam,  
                    validation_steps = int(valSetSize / batch_size),
                    callbacks=[tbCallBack, tbEarly, tbEarly100, tbModelChk])

Epoch 1/30
310/310 [==============================] - 314s 1s/step - loss: 1.7266 - acc: 0.4774 - val_loss: 1.0081 - val_acc: 0.7110
Epoch 2/30
310/310 [==============================] - 271s 873ms/step - loss: 0.6376 - acc: 0.7710 - val_loss: 0.6085 - val_acc: 0.8197
Epoch 3/30
310/310 [==============================] - 379s 1s/step - loss: 0.3778 - acc: 0.8702 - val_loss: 0.4562 - val_acc: 0.8689
Epoch 4/30
310/310 [==============================] - 331s 1s/step - loss: 0.2405 - acc: 0.9186 - val_loss: 0.3398 - val_acc: 0.8984
Epoch 5/30
310/310 [==============================] - 315s 1s/step - loss: 0.1829 - acc: 0.9387 - val_loss: 0.2536 - val_acc: 0.9082
Epoch 6/30
310/310 [==============================] - 348s 1s/step - loss: 0.1224 - acc: 0.9540 - val_loss: 0.2868 - val_acc: 0.9246
Epoch 7/30
310/310 [==============================] - 335s 1s/step - loss: 0.1325 - acc: 0.9500 - val_loss: 0.2643 - val_acc: 0.9311
Epoch 8/30
310/310 [==============================] - 354s 1s/step

# Aplicação do Modelo no Conjunto de Testes

In [11]:
batch_size=4
#Criação do generator p/ o conjunto de teste
testGenerator_resnet = inf619utils.loadDatasetInBatches(test_files, input_shape=input_shape, batch_size=batch_size, nbClasses=nbClasses)

# Load Best weights saved
model.load_weights(filepath='.modelresnet_weights.hdf5')

#Teste
metrics = model.evaluate_generator(testGenerator_resnet, 
                                   steps=int(testSetSize/batch_size), 
                                   verbose=1)

print("Test Loss ResNet---> ", metrics[0])
print("Test Accuracy ResNet---> ", metrics[1])    #Test is balanced, so Acc is normalized

127/127 [==============================] - 15s 115ms/step
Test Loss ResNet--->  0.1663898658770025
Test Accuracy ResNet--->  0.9566929133858267


In [11]:
model.save('resnet_adam_90.h5')
